<h1>Dataset</h1>

In [ ]:
import os
import numpy as np
import pandas as pd
import polars as pl
import re

# Dimensionality Reduction
from scipy.sparse import csr_matrix
import sklearn.feature_extraction.text as sktext
from sklearn.decomposition import PCA, SparsePCA, TruncatedSVD
from sklearn.manifold import TSNE
import umap
import umap.plot

# Clustering
from sklearn.cluster import AgglomerativeClustering, KMeans, SpectralClustering
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import silhouette_samples, silhouette_score
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from yellowbrick.cluster.elbow import kelbow_visualizer

# Plots
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

<h3>Load Data</h3>

In [82]:
demo_stats_df = pl.read_csv("Coursework Data/DemoStats.csv", null_values=None)
household_spend_df = pl.read_csv("Coursework Data/HouseholdSpend.csv", null_values=None)

print(demo_stats_df.describe())
print(household_spend_df.describe())

shape: (9, 737)
┌────────────┬────────┬────────┬──────────┬───┬────────────┬────────────┬────────────┬────────────┐
│ statistic  ┆ CODE   ┆ GEO    ┆ ECYASQKM ┆ … ┆ ECYTCA_18P ┆ ECYNCANCIT ┆ ECYNCA_U18 ┆ ECYNCA_18P │
│ ---        ┆ ---    ┆ ---    ┆ ---      ┆   ┆ ---        ┆ ---        ┆ ---        ┆ ---        │
│ str        ┆ str    ┆ str    ┆ f64      ┆   ┆ f64        ┆ f64        ┆ f64        ┆ f64        │
╞════════════╪════════╪════════╪══════════╪═══╪════════════╪════════════╪════════════╪════════════╡
│ count      ┆ 868970 ┆ 868970 ┆ 868970.0 ┆ … ┆ 868970.0   ┆ 868970.0   ┆ 868970.0   ┆ 868970.0   │
│ null_count ┆ 0      ┆ 0      ┆ 0.0      ┆ … ┆ 0.0        ┆ 0.0        ┆ 0.0        ┆ 0.0        │
│ mean       ┆ null   ┆ null   ┆ 0.0      ┆ … ┆ 32.767185  ┆ 5.353084   ┆ 0.85628    ┆ 4.496803   │
│ std        ┆ null   ┆ null   ┆ 0.0      ┆ … ┆ 130.449848 ┆ 17.893635  ┆ 3.211073   ┆ 15.020402  │
│ min        ┆ A0A0A0 ┆ FSALDU ┆ 0.0      ┆ … ┆ 0.0        ┆ 0.0        ┆ 0.0       

In [83]:
# Define a function to convert NA to None
def convert_na_to_nulls(df):
    return df.with_columns([
        pl.col(col).replace("NA", None).alias(col)
        if df.schema[col] == pl.Utf8 else pl.col(col)
        for col in df.columns
    ])

# Convert NA to None on both dataframes
demo_stats_df = convert_na_to_nulls(demo_stats_df)
household_spend_df = convert_na_to_nulls(household_spend_df)

# Get total nulls for each column
def count_na_strings(df):
    total_rows = df.height
    return {
        col: {
            "null_count": df[col].null_count(),
            "null_percent": round((df[col].null_count() / total_rows) * 100, 2)
        }
        for col in df.columns
        if df[col].null_count() > 0
    }

demo_stats_null_cols = count_na_strings(demo_stats_df)
household_spend_null_cols = count_na_strings(household_spend_df)
# Initial Null Count for each csv file
print("Demo Stats Null Count:", demo_stats_null_cols)
print("Household Stats Null Count:", household_spend_null_cols)

Demo Stats Null Count: {'ECYPTAMED': {'null_count': 87459, 'null_percent': 10.06}, 'ECYPMAMED': {'null_count': 96444, 'null_percent': 11.1}, 'ECYPFAMED': {'null_count': 131837, 'null_percent': 15.17}, 'ECYHTAMED': {'null_count': 92709, 'null_percent': 10.67}, 'ECYHMAMED': {'null_count': 101698, 'null_percent': 11.7}, 'ECYHFAMED': {'null_count': 137900, 'null_percent': 15.87}, 'ECYMTNMED': {'null_count': 92709, 'null_percent': 10.67}}
Household Stats Null Count: {}


In [ ]:
# If you look about at the null counts, you can see that all columns with null counts have more than 1% nulls,
# which means we can drop them. 
demo_stats_df = demo_stats_df.drop(null_cols for null_cols in demo_stats_null_cols)

# Drop irrelevant Identifiers
demo_stats_df = demo_stats_df.drop(["CODE", "GEO"])
household_spend_df = household_spend_df.drop(["CODE","GEO"])

#Impute Missing Values (median or mode depending on data type)
# say why in our report due to null counts (all the same)

In [ ]:
# Get total negative values for each column
def count_negative_values(df):
    return {
        col: {
            "negative_count": df[col].filter(df[col] < 0).len(),
            "negative_percent": round((df[col].filter(df[col] < 0).len() / df.height) * 100, 2)
        }
        for col in df.columns
        if df.schema[col] in [pl.Int8, pl.Int16, pl.Int32, pl.Int64,
                              pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64,
                              pl.Float32, pl.Float64]
        and df[col].filter(df[col] < 0).len() > 0
    }

# Get total negative values for each dataframe
demo_stats_negatives = count_negative_values(demo_stats_df)
household_spend_negatives = count_negative_values(household_spend_df)

# Initial Negative Count for each csv file
print("Demo Stats Negative Values:", demo_stats_negatives)
print("Household Spend Negative Values:", household_spend_negatives)

Demo Stats Negative Values: {}
Household Spend Negative Values: {'HSTT001': {'negative_count': 28, 'negative_percent': 0.0}, 'HSTE001ZBS': {'negative_count': 163557, 'negative_percent': 18.82}, 'HSWH040S': {'negative_count': 255858, 'negative_percent': 29.44}, 'HSWH041S': {'negative_count': 15436, 'negative_percent': 1.78}, 'HSWH042S': {'negative_count': 19205, 'negative_percent': 2.21}}


In [ ]:
#HSTT001 - should not be negative as it describes total amount of money spent (clarify)

# HSTE001ZBS - Total non-current consumption,Household Expenditures (Category Summary),Dollars
'''- spending on long-term goods/services — such as:

- Durable goods (cars, appliances)

- Real estate

- Long-term services or investments

'''


# HSWH041S - Net purchase price of owned secondary residences,Household Expenditures (Category Summary),Dollars

# HSWH042S - Net purchase price of other owned properties,Household Expenditures (Category Summary),Dollars

In [ ]:
#merge datasets


#standardize?

In [77]:
# Redisplay descriptive statistics
print(demo_stats_df.describe())
print(household_spend_df.describe())

shape: (9, 728)
┌───────────┬──────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ statistic ┆ ECYASQKM ┆ ECYALSQKM ┆ ECYBASPOP ┆ … ┆ ECYTCA_18 ┆ ECYNCANCI ┆ ECYNCA_U1 ┆ ECYNCA_18 │
│ ---       ┆ ---      ┆ ---       ┆ ---       ┆   ┆ P         ┆ T         ┆ 8         ┆ P         │
│ str       ┆ f64      ┆ f64       ┆ f64       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│           ┆          ┆           ┆           ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64       │
╞═══════════╪══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ count     ┆ 868970.0 ┆ 868970.0  ┆ 868970.0  ┆ … ┆ 868970.0  ┆ 868970.0  ┆ 868970.0  ┆ 868970.0  │
│ null_coun ┆ 0.0      ┆ 0.0       ┆ 0.0       ┆ … ┆ 0.0       ┆ 0.0       ┆ 0.0       ┆ 0.0       │
│ t         ┆          ┆           ┆           ┆   ┆           ┆           ┆           ┆           │
│ mean      ┆ 0.0      ┆ 0.0       ┆ 46.684199 ┆ … ┆ 32.767185 ┆ 5.353084  

<h1>Part 1: Clustering and Dimensionality Reduction</h1>

The first part of the coursework will focus
on identifying the characteristics of Canadian households, excluding their pension
behaviour. For this, do not include, in your clustering and dimensionality reduction models,
the target of the regression model in Part 2.

Clean the data using your knowledge of the course. For context, you will
create in later questions a model that predicts a household’s proportion of income
spent on total personal insurance premiums and retirement/pension contributions
and apply a clustering algorithm over the full data.

In [78]:
# Merge dataset, but separate target

# Deal with null values, outliers, dirty values, inconsistent values (GIGO)

Create a K-Means clustering of the data, identifying the optimal number of
clusters using both the silhouette and the elbow method. Do they agree?

In [79]:
# Initialize KClusterer

# Use KElbowVisualizer to find optimal number of clusters

# Use Silhouette to find optimal number of clusters

# Compare results and determine optimal number of clusters

Now we will apply a linear dimensionality reductions technique to the data.
1. Apply PCA to your data.
2. Plot the data of the first two PCs in a scatterplot and colour the points as
per the cluster labels you calculated in the previous step. What can you say
about your data? Are your clusters clearly defined in the output? Interpret
the first three components of your PCA output.
3. For the first three components, calculate the average value of each
component, for each cluster, so your data should look like a table with
cluster number and average component value. Give a name to the clusters
from this output and justify your choice.

 Now, use UMAP to reduce the data to two dimensions. Justify your choice of
parameters by searching for the optimal value as you deem reasonable. Again,
colour the data to differentiate each cluster that you named. What do you see? Is
your UMAP a better or worse interpretation than PCA?

<h1>Part 2: Regression</h1>

Now we will create models for a household’s proportion of income spent
on total personal insurance premiums and retirement/pension contributions.

Train a regularized elastic net linear regression from your data.

1. Create your target variable from the variables in the dataset. Do not use
those components on the training database.
2. Apply any data transformation / variable creation you deem necessary to
obtain a good result.
3. Discuss the grid that you chose to search for the parameters and the output
that you obtained.
4. For your test set, create a scatterplot of the original response and the
predicted response. Report the MSE and R2 on the test set and calculate a
bootstrapped confidence interval of the output.
5. Interpret the coefficients of the top five most important variables in the
regression